# 豆瓣美剧

## 只根据Tag界面筛选

### Improvement

1. 登录与非登录状态，豆瓣美剧内容显示不相同，所以该方法会有很多重复信息
   -> python 去重 还是 存入数据库 再去重 (毕竟信息不多)
   -> Spider中 Cookies 如何保存
2. Scrapy框架等的运用
   
### Progress
Oct. 7:<br/>
提取了title并插入数据库，可用。然而...爬取太快然后403 Forbidden，估计封了IP -> 不设间隔时间的爬虫都是耍流氓...<br/> 

In [74]:
import re

class Tools:
    # 去除空格
    removeBlock = re.compile('\n+')
    def replace(self, x):
        x = re.sub(removeBlock, "", x)
        return x.strip()

In [1]:
import urllib
import urllib2
import requests
import re
from lxml import etree
import time
import MySQLdb

In [2]:
def GetPageNum(page):
    dom = etree.HTML(page)
    num = dom.xpath('//div[@class="paginator"]/a[last()]/text()')
    if len(num) > 0:
        return int(num[0])
    else:
        return 0

In [3]:
def replace(x):
    removeBlock = re.compile('\n|\n+| {1,}|<br>|&nbsp;|', re.S)
    removeBackSlash = re.compile('\/')
    x = re.sub(removeBackSlash, "", x)
    x = re.sub(removeBlock, "", x)
    return x.strip()

In [4]:
def GetContent(page, rank, people):
    dom = etree.HTML(page)
    titles = dom.xpath('//div[@class="pl2"]/a/text()')
#     print allContent
    results = []
#     print page
    for title in titles:
        result = replace(title)
#         print result
        if isinstance(result, unicode) == True:
            tmp_list = []
            tmp_list.append(result)
#             print tmp_list
            results.append(tmp_list)
    return results
#     print results
#         print 
        
#     //message[@id=0]//books/*[local-name()='book']

In [15]:
def WriteData(content):
    conn = MySQLdb.connect(host="localhost", user="ycy", passwd="lyc1222", db="spider", charset="utf8")
    cursor = conn.cursor()
    
#     # 删除表
#     sql = "drop table if exists movie"
#     cursor.execute(sql)
    
    # 创建表
    sql = """create table if not exists movie
       (id int(10) auto_increment primary key, name varchar(1000), rank float, people int(10))
       DEFAULT CHARSET=utf8"""
    cursor.execute(sql)
    
#     # 测试用
#     sql = 'insert into movie(name) values("测试")'
#     n = cursor.execute(sql)
    
    # 写入多行
    sql = "insert into movie(name) values(%s)"
    n = cursor.executemany(sql, content)
    print 'insertmany', n
    
    cursor.close()
    conn.commit()
    conn.close()

In [13]:
def Spider(url, rank, people):
    i = 0
    print "Updating... " 
    myPage = requests.get(url).content.decode("utf-8")
#     print myPage
    pageNum = GetPageNum(myPage)
    content = []
    print pageNum
    
    for i in range(pageNum):
        time.sleep(60)
        search_url = url + u"?start=" + str(len(content)).decode("utf-8") + "&type=T"
#         search_url = url + u"?start=" + str(20).decode("utf-8") + "&type=T"
        print search_url
#         user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
#         headers = {'User-Agent': user_agent}
#         request = urllib2.Request(search_url, headers) 
#         response = urllib2.urlopen(request)
#         myPage = response.read().decode("utf-8")
        myPage = requests.get(search_url).content.decode("utf-8")
        tmp_content = GetContent(myPage, rank, people)
        WriteData(tmp_content)
        content.extend(tmp_content)
#     print content
        print str(len(content)) + u"---------------------------------------------------------"
        
#     print type(content[1])
#     WriteData(content)
#     print len(content)
#     print pageNum
#     print type(pageNum)
#     //message[@id=0]//books/*[local-name()='book']

In [16]:
if __name__ == '__main__':
    print "start"
    start_url = "https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7"
#     rank = raw_input("请输入最低评分限制：\n")
#     people = raw_input("请输入最低评分人数：\n")
    rank = 8
    people = 10000
    Spider(start_url, rank, people)
    print "end"

start
Updating... 
146
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=0&type=T
insertmany 20
20---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=20&type=T


/Users/ycymio/Documents/Python/venv/lib/python2.7/site-packages/ipykernel/__main__.py:13: Warning: Table 'movie' already exists


insertmany 19
39---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=39&type=T
insertmany 19
58---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=58&type=T
insertmany 20
78---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=78&type=T
insertmany 16
94---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=94&type=T
insertmany 17
111---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=111&type=T
insertmany 19
130---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=130&type=T
insertmany 20
150---------------------------------------------------------
https://movie.douban.com/tag/%E7%BE%8E%E5%89%A7?start=150&type=T
insertmany 20
170-----------

KeyboardInterrupt: 